In [1]:
from environments.pickplace_environment_residual import ResPickOrPlaceEnvWithoutLangReward
from tasks.letter import PutLetterOontheBowl
import numpy as np
import matplotlib.pyplot as plt
from agents.LLMRL import LLMSAC,GuideSAC
import cv2

pybullet build time: May 20 2022 19:44:17
2023-10-31 16:28:34.090595: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-31 16:28:34.122043: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 16:28:35.121286: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from PIL import Image
import torch
from reward.detector import VILD


In [3]:
vild = VILD()
print(vild.session.list_devices())

Instructions for updating:
Use `tf.saved_model.load` instead.


2023-10-31 16:28:39.951424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-31 16:28:39.952428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-31 16:28:39.952555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

INFO:tensorflow:Restoring parameters from ./reward/image_path_v2/variables/variables


2023-10-31 16:28:40.714005: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled


[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 5831299169101005792), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 3467378688, 7535381175147390662)]


In [22]:
def match_bbox_category(found_obj,scores,category):
    scores = scores[:,1:]
    unoccupied_cata = np.arange(len(category))
    unoccupied_obj = np.arange(len(found_obj))
    new_found_obj = []
    while len(unoccupied_cata) > 0 and len(unoccupied_obj) > 0:
        mask = np.zeros_like(scores)
        mask[np.ix_(unoccupied_obj,unoccupied_cata)] = 1
        cur_score =  np.where(mask,scores,0)
        # get max score position
        max_idx = np.unravel_index(np.argmax(cur_score, axis=None), cur_score.shape)
        obj = found_obj[max_idx[0]]
        bbox = obj[2]
        w,h = bbox[2]-bbox[0],bbox[3]-bbox[1]
        area = w*h
        if area > 6000 or w>100 or h>100:
            unoccupied_obj = unoccupied_obj[unoccupied_obj!=max_idx[0]]
            continue
        else:
            obj = (category[max_idx[1]],found_obj[max_idx[0]][1],found_obj[max_idx[0]][2],scores[max_idx[0],max_idx[1]])
            new_found_obj.append(obj)
            # remove the row and column
            # print('remove',max_idx,cur_score[max_idx[0],max_idx[1]])
            unoccupied_obj = unoccupied_obj[unoccupied_obj!=max_idx[0]]
            unoccupied_cata = unoccupied_cata[unoccupied_cata!=max_idx[1]]
    return new_found_obj
def visuallizre_res(img,found_obj,scores):
    for obj in found_obj:
        if obj[0].split(" ")[1] == "bowl":
            rec_color =(0, 255, 0)
        else:
            rec_color =(0, 0, 255)
        y_min,x_min,y_max,x_max = obj[2]
        img = cv2.rectangle(img,(x_min, y_min), (x_max, y_max),rec_color , 2,)
        img = cv2.putText(img, obj[0], (x_min, (y_min+y_max)//2), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (255, 0, 0), 1, cv2.LINE_AA)
    plt.imshow(img)
    plt.show()
    # plot matrix score_all
    plt.imshow(scores, cmap='viridis')  # 'cmap' parameter defines the colormap
    plt.colorbar() 
    m,n = scores.shape
    for i in range(m):
        for j in range(n):
            text = plt.text(j, i, f'{scores[i, j]:.2f}',
                            ha="center", va="center", color="w")
    plt.show()
    for obj in found_obj:
        print(obj)


     
    

In [23]:
import ipywidgets
env = ResPickOrPlaceEnvWithoutLangReward(
                                        task= PutLetterOontheBowl,
                                        image_obs=True,
                                        residual=True,
                                        observation_noise=5,
                                        render=False,
                                        multi_discrete=False,
                                        scale_action=True,
                                        ee="suction",
                                        scale_obs=True,
                                        neglect_steps=False,
                                        one_hot_action = True)
def show_image(seed=1):
    np.random.seed(seed)
    env.reset()
    img = env.get_image()

    image_path = "tmp/images/PutLetterOontheBowl_step0_seed{}.png".format(seed)
    image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_path, image)

    catagory = [
        "yellow bowl","green bowl","blue bowl",
        "yellow o shape block","blue o shape block", "green o shape block"]
    found_obj,scores,image_det = vild.vild_detect(image_path,catagory,verbose= False)
    found_obj = match_bbox_category(found_obj,scores,catagory)
    
                        
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    visuallizre_res(img,found_obj,scores)
    
ipywidgets.interact(show_image, seed=(0, 10, 1))

interactive(children=(IntSlider(value=1, description='seed', max=10), Output()), _dom_classes=('widget-interac…

<function __main__.show_image(seed=1)>

In [2]:

model =LLMSAC.load("tmp/llmsac_imgatten_withnoise15.0fixedputblockbowl_model/rl_model_30000_steps.zip")



/media/marunyu/Data/study/DELFT/master_thesis/RA/venv/lib/python3.8/site-packages/stable_baselines3/common/buffers.py:590: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 12.05GB > 3.91GB
  warnings.warn(


In [4]:
env = ResPickOrPlaceEnvWithoutLangReward(
                                        task= PutLetterOontheBowl,
                                         image_obs=True,
                                         residual=True,
                                         observation_noise=5,
                                         render=True,
                                         multi_discrete=False,
                                         scale_action=True,
                                         ee="suction",
                                         scale_obs=True,
                                         neglect_steps=False,
                                      one_hot_action = True)
success = 0# 97%
trial = 100
steps = []
#step2 0.79
# step3 0.12
# step4 0.03
for seed in range(trial):
    np.random.seed(seed)
    obs,_ = env.reset()
    done = False
    step = 0
    while not done:
        action,_ = model.predict(obs, deterministic=True)
        print(action)
        obs, rewards, done,_, info = env.step(action)
        step += 1
        if done:
            steps.append(step)
            if rewards >= 0.95:
                success += 1
            print("done")
            break
RL_success = success/trial
env.close()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) UHD Graphics (TGL GT1)
GL_VERSION=4.6 (Core Profile) Mesa 23.0.4-0ubuntu1~22.04.1
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.0.4-0ubuntu1~22.04.1
Vendor = Intel
Renderer = Mesa Intel(R) UHD Graphics (TGL GT1)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
init scene
obj_ids [6, 7, 8]
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL dr

In [9]:
step2 = steps.count(2)/len(steps)
step3 = steps.count(3)/len(steps)
step4 = steps.count(4)/len(steps)
print("step2",step2)
print("step3",step3)
print("step4",step4)

step2 0.79
step3 0.12
step4 0.03


In [3]:
import ipywidgets
env = ResPickOrPlaceEnvWithoutLangReward(
                                        task= PutLetterOontheBowl,
                                         image_obs=True,
                                         residual=True,
                                         observation_noise=5,
                                         render=True,
                                         multi_discrete=False,
                                         scale_action=True,
                                         ee="suction",
                                         scale_obs=True,
                                         neglect_steps=False,
                                         one_hot_action = True)
success = 0# 97%
trial = 100
steps = []
# RL_success 0.02
# step2 0.0
# step3 0.0
# step4 0.01
# step10 0.0
# step11 0.98
for seed in range(trial):
    np.random.seed(seed)
    obs,_ = env.reset()
    done = False
    step = 0
    while not done:
        action = env.get_expert_demonstration()
        action[-2:]=[0,0]
        print(action)
        obs, rewards, done,_, info = env.step(action)
        step += 1
        if done:
            steps.append(step)
            if rewards >= 0.95:
                success += 1
            print("done")
            break
RL_success = success/trial
env.close()


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) UHD Graphics (TGL GT1)
GL_VERSION=4.6 (Core Profile) Mesa 23.0.4-0ubuntu1~22.04.1
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.0.4-0ubuntu1~22.04.1
Vendor = Intel
Renderer = Mesa Intel(R) UHD Graphics (TGL GT1)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
init scene
obj_ids [6, 7, 8]
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL dr

In [6]:
print("RL_success",RL_success)
step2 = steps.count(2)/len(steps)
step3 = steps.count(3)/len(steps)
step4 = steps.count(4)/len(steps)
step10 = steps.count(10)/len(steps)
step11 = steps.count(11)/len(steps)
print("step2",step2)
print("step3",step3)
print("step4",step4)
print("step10",step10)
print("step11",step11)

RL_success 0.02
step2 0.0
step3 0.0
step4 0.01
step10 0.0
step11 0.98


: 